# MED-PC Extracting the Recording Data and Metadata

## Importing the Python Libraries

In [1]:
import sys
import glob
from collections import defaultdict
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from moviepy.editor import *
from datetime import datetime
from IPython.display import Video

In [2]:
from medpc2excel.medpc_read import medpc_read

In [3]:
# setting path
sys.path.append('../../src')

In [4]:
# All the libraries that were created for this repository
import extract.dataframe
import processing.tone
import extract.metadata

## Getting the Metadata from all the MED-PC Recording Files

- Getting all the file paths of the recording files(that happen to all end in `.txt`

# NOTE: The following path must be changed to the directory where your MED-PC recording files are located, if they are not in the specied folder

In [175]:
all_med_pc_file = glob.glob("./data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/*.txt")

- Use this instead if you're using you're own data

In [176]:
all_med_pc_file[:10]

['./data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-22_12h26m_Subject 1.txt',
 './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-23_14h55m_Subject 4.txt',
 './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-23_10h06m_Subject 2.txt',
 './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-26_11h36m_Subject 4.txt',
 './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-22_10h11m_Subject 2.txt',
 './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-22_09h00m_Subject 3.txt',
 './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-26_09h05m_Subject 3.txt',
 './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-22_11h19m_Subject 3.txt',
 './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-23_14h55m_Subject 3.txt',
 './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-19_13h38m_Subject 1.txt']

- Example of what the MED-PC Recording file looks like

In [177]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-07-22_12h26m_Subject 1.txt







Start Date: 07/22/22

End Date: 07/22/22

Subject: 1

Experiment: C57_vs_CD1_comparison 

Group: Cage4

Box: 1

Start Time: 12:26:57

End Time: 13:36:44

MSN: CD1_reward_training

A:    4399.000

D:    9000.000

E:       0.000

L:       0.000

M:       0.000

O:       0.000

T:    3660.000



- We will be extracting the first 10 or so lines that look like:

```
File: C:\MED-PC\Data\2022-05-06_12h59m_Subject 3.4 (2).txt

Start Date: 05/06/22

End Date: 05/06/22

Subject: 3.4 (2)

Experiment: Pilot of Pilot

Group: Cage 4

Box: 1

Start Time: 12:59:58

End Time: 14:02:38

MSN: levelNP_CS_reward_laserepochON1st_noshock
```
    
- We will just find all the lines that start with "File", "Start Date", "End Date", "Subject", "Experiment", "Group", "Box", "Start Time", "End Time", or "MSN". And then stop once all the metadata types have been collected

In [178]:
# This makes a nested dictionary of file paths to each individual metadata type
file_path_to_meta_data = extract.metadata.get_all_med_pc_meta_data_from_files(list_of_files=all_med_pc_file)

In [179]:
for key, value in file_path_to_meta_data.items():
    print("File path: {}".format(key))
    print("Metadata types and associated values: {}".format(value))
    break

File path: ./data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-22_12h26m_Subject 1.txt
Metadata types and associated values: {'File': 'C:\\MED-PC\\Data\\2022-07-22_12h26m_Subject 1.txt', 'Start Date': '07/22/22', 'End Date': '07/22/22', 'Subject': '1', 'Experiment': 'C57_vs_CD1_comparison', 'Group': 'Cage4', 'Box': '1', 'Start Time': '12:26:57', 'End Time': '13:36:44', 'MSN': 'CD1_reward_training'}


## Making a Dataframe out of the Metadata

In [180]:
# Turning the dictionary into a Pandas Dataframe
metadata_df = pd.DataFrame.from_dict(file_path_to_meta_data, orient="index")
# Resetting the index because currently the file path is the index 
metadata_df = metadata_df.reset_index()

In [181]:
metadata_df.head()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
0,./data/pilot_c57_vs_cd1/reward_training/med_pc...,C:\MED-PC\Data\2022-07-22_12h26m_Subject 1.txt,07/22/22,07/22/22,1,C57_vs_CD1_comparison,Cage4,1,12:26:57,13:36:44,CD1_reward_training
1,./data/pilot_c57_vs_cd1/reward_training/med_pc...,C:\MED-PC\Data\2022-07-23_14h55m_Subject 4.txt,07/23/22,07/23/22,4,C57_vs_CD1_Comparison,Cage 6,1,14:55:47,16:05:03,CD1_reward_training
2,./data/pilot_c57_vs_cd1/reward_training/med_pc...,C:\MED-PC\Data\2022-07-23_10h06m_Subject 2.txt,07/23/22,07/23/22,2,C57_vs_CD1_comparison,Cage2,3,10:06:22,11:09:48,CD1_reward_training
3,./data/pilot_c57_vs_cd1/reward_training/med_pc...,C:\MED-PC\Data\2022-07-26_11h36m_Subject 4.txt,07/26/22,07/26/22,4,C57_vs_CD1_Comparison,Cage3,4,11:36:00,12:41:44,C57_reward_training
4,./data/pilot_c57_vs_cd1/reward_training/med_pc...,C:\MED-PC\Data\2022-07-22_10h11m_Subject 2.txt,07/22/22,07/22/22,2,C57_vs_CD1_comparison,Cage2,2,10:11:03,11:14:37,CD1_reward_training


In [182]:
metadata_df.tail()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
212,./data/pilot_c57_vs_cd1/reward_training/med_pc...,C:\MED-PC\Data\2022-07-22_10h11m_Subject 1.txt,07/22/22,07/22/22,1,C57_vs_CD1_comparison,Cage2,1,10:11:03,11:14:37,C57_reward_training
213,./data/pilot_c57_vs_cd1/reward_training/med_pc...,C:\MED-PC\Data\2022-07-26_11h36m_Subject 2.txt,07/26/22,07/26/22,2,C57_vs_CD1_Comparison,Cage3,2,11:36:00,12:41:44,C57_reward_training
214,./data/pilot_c57_vs_cd1/reward_training/med_pc...,C:\MED-PC\Data\2022-07-20_10h48m_Subject 3.txt,07/20/22,07/20/22,3,C57_vs_CD1_Comparison,Cage3,1,10:48:32,11:53:40,C57_reward_training
215,./data/pilot_c57_vs_cd1/reward_training/med_pc...,C:\MED-PC\Data\2022-07-25_14h43m_Subject 2.txt,07/25/22,07/25/22,2,C57_vs_CD1_Comparison,cage6,1,14:43:50,15:46:38,CD1_reward_training
216,./data/pilot_c57_vs_cd1/reward_training/med_pc...,C:\MED-PC\Data\2022-07-24_14h53m_Subject 3.txt,07/24/22,07/24/22,3,C57_vs_CD1_comparison,Cage 6,1,14:53:31,15:56:03,CD1_reward_training


- Just getting the numbers out of the column that contains the cage information

## Inputting all the MED-PC log files

- Example of what the MED-PC Script looks like that was ran when recording the behaviors

In [183]:
with open("./scripts/levelNP_CS_reward_laserepochON1st_noshock.MPC") as f:
    lines = f.readlines()
    for line in lines[:100]:
        print(line)

\v3 stop tone with poke

\v3.2 monitor port entries AND exits



\INPUTS

^port = 8



\OUTPUTS

^fan = 16

^houselight = 11

^tone1 = 2

^tone2 = 3

^tone3 = 4

^tone4 = 5

^pump = 9

^whitenoise = 1

^laser=10

^csout = 5

^peout = 15

^cs1out = 12

^cs2out = 13

^cs3out = 14



\EXP SETTINGS

^ncsNoShock = 0

^initCS1trials = 3



\ARRAYS

DIM P = 20000 \Port entry time stamp array

DIM Q = 2500 \US delivery time stamp array (absolute)

DIM R = 2500 \US time stamp array (relative to last CS)

DIM W = 2500 \ITI values used for CS

DIM S = 2500 \CS presentation values (absolute - every time light turns on)

DIM N = 20000 \Port exit time stamp array

DIM K = 2500 \CS type

DIM B = 2500 \shock intensity



LIST V = 30", 30", 30", 30", 100", 95", 90", 80", 90", 100", 90", 120", 90", 85", 90", 95", 120", 80", 95", 80", 90", 80", 75", 100", 95", 90", 80", 90", 100", 90", 120", 90", 85", 90", 95", 90", 80", 90", 100", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90"

- We will be using the comments in the MED-PC script(which starts with the `\`) to create a name for the variables. By default, MED-PC uses a single letter as the name of the variable.
    - This will use the medpc2excel library found in https://github.com/cyf203/medpc2excel
- Example of the comments in the MED-PC script that we will use the names from:

```
\Variables

\A - Time since last CS

\B - Shock intensity

\C - Counter array

\D - Current ITI value

\E - CS ITI values for first few trials

\F - Shock duration

\G -

\H -

\I -

\J - Shock intensity repo

\K - CS type

\L -

\M - CS type repo

\N - Port exit time stamp array

\O -

\P - Port entry time stamp array

\Q - Sucrose delivery time stamp array (absolute)

\R - Sucrose delivery time stamp array (relative to last CS)

\S - CS presentation time stamp array

\T - Session timer

\U - Time since last CS presentation

\V - List of CS ITI values (tone + houselight)

\W - ITI values used for CS one each trial

\X -

\Y - Beam break monitor variable

\Z -
```

In [184]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-07-22_12h26m_Subject 1.txt







Start Date: 07/22/22

End Date: 07/22/22

Subject: 1

Experiment: C57_vs_CD1_comparison 

Group: Cage4

Box: 1

Start Time: 12:26:57

End Time: 13:36:44

MSN: CD1_reward_training

A:    4399.000

D:    9000.000

E:       0.000

L:       0.000

M:       0.000

O:       0.000

T:    3660.000



- **Please make sure that the corresponding `.mpc` file (aka the MED-PC script) that was ran to create the log file, is also in the same folder**

In [185]:
concatted_medpc_df = extract.dataframe.get_medpc_dataframe_from_list_of_files(medpc_files=all_med_pc_file)
concatted_medpc_df = concatted_medpc_df.reset_index(drop=True)

Traceback (most recent call last):
  File "/root/projects/behavioral_dataframe_processing/results/2022_07_27_shorter_video_investigation/../../src/extract/dataframe.py", line 71, in get_medpc_dataframe_from_list_of_files
    ts_df, medpc_log = medpc_read(file=file_path, override=True, replace=False)
  File "/root/projects/behavioral_dataframe_processing/behavioral_processing_env/lib/python3.9/site-packages/medpc2excel/medpc_read.py", line 134, in medpc_read
    for var, nm in TS_var_name_maps[program_nm].items():
KeyError: 'pumptest'

Invalid Formatting for file: ./data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-21_12h24m_Subject 0.txt


In [186]:
concatted_medpc_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
0,6.22,64.0,399.0,0.0,60.01,6.26,1.0,44.0,36.0,12.0,50.0,35.0,12.0,20220722,1,./data/pilot_c57_vs_cd1/reward_training/med_pc...
1,6.28,144.0,399.0,0.0,140.01,6.36,1.0,4.0,38.0,12.0,50.0,35.0,12.0,20220722,1,./data/pilot_c57_vs_cd1/reward_training/med_pc...
2,6.38,234.0,399.0,0.0,230.01,6.41,1.0,34.0,39.0,12.0,50.0,35.0,12.0,20220722,1,./data/pilot_c57_vs_cd1/reward_training/med_pc...
3,6.44,314.0,399.0,0.0,310.01,6.52,1.0,54.0,40.0,12.0,51.0,35.0,12.0,20220722,1,./data/pilot_c57_vs_cd1/reward_training/med_pc...
4,6.64,389.0,399.0,0.0,385.01,6.69,1.0,9.0,42.0,12.0,51.0,35.0,12.0,20220722,1,./data/pilot_c57_vs_cd1/reward_training/med_pc...


In [187]:
concatted_medpc_df.tail()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
1356741,3651.72,NaN,NaN,NaN,NaN,3651.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220724,3,./data/pilot_c57_vs_cd1/reward_training/med_pc...
1356742,3651.81,NaN,NaN,NaN,NaN,3651.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220724,3,./data/pilot_c57_vs_cd1/reward_training/med_pc...
1356743,3655.18,NaN,NaN,NaN,NaN,3655.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220724,3,./data/pilot_c57_vs_cd1/reward_training/med_pc...
1356744,3655.31,NaN,NaN,NaN,NaN,3655.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220724,3,./data/pilot_c57_vs_cd1/reward_training/med_pc...
1356745,3655.42,NaN,NaN,NaN,NaN,3655.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220724,3,./data/pilot_c57_vs_cd1/reward_training/med_pc...


- Dropping all rows that don't have the CS presentation time

In [287]:
cs_time_df = concatted_medpc_df[concatted_medpc_df["(J)port_entry_hourscomputer"] != 0.0].dropna(subset="(J)port_entry_hourscomputer")

In [288]:
cs_time_df["file_path"].unique()

array(['./data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-22_12h26m_Subject 1.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-23_14h55m_Subject 4.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-23_10h06m_Subject 2.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-26_11h36m_Subject 4.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-22_10h11m_Subject 2.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-22_09h00m_Subject 3.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-26_09h05m_Subject 3.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-22_11h19m_Subject 3.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-23_14h55m_Subject 3.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-19_13h38m_Subject 1.txt',


# Making a column for the Controlle Stimulus time

- Because the hour, the minute, and the seconds are all in different columns we will combine them into one

In [289]:
cs_time_df.columns

Index(['(P)Portentry', '(Q)USdelivery', '(R)UStime', '(W)ITIvalues',
       '(S)CSpresentation', '(N)Portexit', '(K)CStype',
       '(G)controlled_stimulus_secondscomputer',
       '(H)controlled_stimulus_minutescomputer',
       '(I)controlled_stimulus_hourscomputer', '(B)port_entry_secondscomputer',
       '(F)port_entry_minutescomputer', '(J)port_entry_hourscomputer', 'date',
       'subject', 'file_path'],
      dtype='object')

- Making the numbers for hours, minutes, and seconds into whole numbers

In [290]:
cs_time_df["cs_second"] = cs_time_df["(G)controlled_stimulus_secondscomputer"].astype(int).astype(str)
cs_time_df["cs_minute"] = cs_time_df["(H)controlled_stimulus_minutescomputer"].astype(int).astype(str)
cs_time_df["cs_hour"] = cs_time_df["(I)controlled_stimulus_hourscomputer"].astype(int).astype(str)


- Combining the hours, minutes, and seconds into one column as a string

In [291]:
cs_time_df["cs_time_str"] = cs_time_df.apply(lambda x: ":".join([x["cs_hour"], x["cs_minute"],  x["cs_second"]]), axis=1)

- Converting the time into a Python datetime object
    - This will allow us to make calculations

In [292]:
cs_time_df["cs_time_object"] = cs_time_df["cs_time_str"].apply(lambda x: datetime.strptime(x, '%H:%M:%S'))


In [293]:
cs_time_df

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,...,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path,cs_second,cs_minute,cs_hour,cs_time_str,cs_time_object
0,6.22,64.0,399.0,0.0,60.01,6.26,1.0,44.0,36.0,12.0,...,35.0,12.0,20220722,1,./data/pilot_c57_vs_cd1/reward_training/med_pc...,44,36,12,12:36:44,1900-01-01 12:36:44
1,6.28,144.0,399.0,0.0,140.01,6.36,1.0,4.0,38.0,12.0,...,35.0,12.0,20220722,1,./data/pilot_c57_vs_cd1/reward_training/med_pc...,4,38,12,12:38:4,1900-01-01 12:38:04
2,6.38,234.0,399.0,0.0,230.01,6.41,1.0,34.0,39.0,12.0,...,35.0,12.0,20220722,1,./data/pilot_c57_vs_cd1/reward_training/med_pc...,34,39,12,12:39:34,1900-01-01 12:39:34
3,6.44,314.0,399.0,0.0,310.01,6.52,1.0,54.0,40.0,12.0,...,35.0,12.0,20220722,1,./data/pilot_c57_vs_cd1/reward_training/med_pc...,54,40,12,12:40:54,1900-01-01 12:40:54
4,6.64,389.0,399.0,0.0,385.01,6.69,1.0,9.0,42.0,12.0,...,35.0,12.0,20220722,1,./data/pilot_c57_vs_cd1/reward_training/med_pc...,9,42,12,12:42:9,1900-01-01 12:42:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356432,2946.85,NaN,NaN,0.0,NaN,2947.61,0.0,0.0,0.0,0.0,...,44.0,15.0,20220724,3,./data/pilot_c57_vs_cd1/reward_training/med_pc...,0,0,0,0:0:0,1900-01-01 00:00:00
1356433,2947.63,NaN,NaN,0.0,NaN,2947.88,0.0,0.0,0.0,0.0,...,44.0,15.0,20220724,3,./data/pilot_c57_vs_cd1/reward_training/med_pc...,0,0,0,0:0:0,1900-01-01 00:00:00
1356434,2947.93,NaN,NaN,0.0,NaN,2948.09,0.0,0.0,0.0,0.0,...,44.0,15.0,20220724,3,./data/pilot_c57_vs_cd1/reward_training/med_pc...,0,0,0,0:0:0,1900-01-01 00:00:00
1356435,2948.11,NaN,NaN,0.0,NaN,2949.04,0.0,0.0,0.0,0.0,...,44.0,15.0,20220724,3,./data/pilot_c57_vs_cd1/reward_training/med_pc...,0,0,0,0:0:0,1900-01-01 00:00:00


# Making a column for the Port Entry time

- Because the hour, the minute, and the seconds are all in different columns we will combine them into one

In [294]:
cs_time_df.columns

Index(['(P)Portentry', '(Q)USdelivery', '(R)UStime', '(W)ITIvalues',
       '(S)CSpresentation', '(N)Portexit', '(K)CStype',
       '(G)controlled_stimulus_secondscomputer',
       '(H)controlled_stimulus_minutescomputer',
       '(I)controlled_stimulus_hourscomputer', '(B)port_entry_secondscomputer',
       '(F)port_entry_minutescomputer', '(J)port_entry_hourscomputer', 'date',
       'subject', 'file_path', 'cs_second', 'cs_minute', 'cs_hour',
       'cs_time_str', 'cs_time_object'],
      dtype='object')

- Making the numbers for hours, minutes, and seconds into whole numbers

In [295]:
cs_time_df["pe_second"] = cs_time_df["(B)port_entry_secondscomputer"].astype(int).astype(str)
cs_time_df["pe_minute"] = cs_time_df["(F)port_entry_minutescomputer"].astype(int).astype(str)
cs_time_df["pe_hour"] = cs_time_df["(J)port_entry_hourscomputer"].astype(int).astype(str)


- Combining the hours, minutes, and seconds into one column as a string

In [296]:
cs_time_df["pe_time_str"] = cs_time_df.apply(lambda x: ":".join([x["pe_hour"], x["pe_minute"],  x["pe_second"]]), axis=1)

- Converting the time into a Python datetime object
    - This will allow us to make calculations

In [297]:
cs_time_df["pe_time_object"] = cs_time_df["pe_time_str"].apply(lambda x: datetime.strptime(x, '%H:%M:%S'))


In [298]:
cs_time_df

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,...,cs_second,cs_minute,cs_hour,cs_time_str,cs_time_object,pe_second,pe_minute,pe_hour,pe_time_str,pe_time_object
0,6.22,64.0,399.0,0.0,60.01,6.26,1.0,44.0,36.0,12.0,...,44,36,12,12:36:44,1900-01-01 12:36:44,50,35,12,12:35:50,1900-01-01 12:35:50
1,6.28,144.0,399.0,0.0,140.01,6.36,1.0,4.0,38.0,12.0,...,4,38,12,12:38:4,1900-01-01 12:38:04,50,35,12,12:35:50,1900-01-01 12:35:50
2,6.38,234.0,399.0,0.0,230.01,6.41,1.0,34.0,39.0,12.0,...,34,39,12,12:39:34,1900-01-01 12:39:34,50,35,12,12:35:50,1900-01-01 12:35:50
3,6.44,314.0,399.0,0.0,310.01,6.52,1.0,54.0,40.0,12.0,...,54,40,12,12:40:54,1900-01-01 12:40:54,51,35,12,12:35:51,1900-01-01 12:35:51
4,6.64,389.0,399.0,0.0,385.01,6.69,1.0,9.0,42.0,12.0,...,9,42,12,12:42:9,1900-01-01 12:42:09,51,35,12,12:35:51,1900-01-01 12:35:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356432,2946.85,NaN,NaN,0.0,NaN,2947.61,0.0,0.0,0.0,0.0,...,0,0,0,0:0:0,1900-01-01 00:00:00,10,44,15,15:44:10,1900-01-01 15:44:10
1356433,2947.63,NaN,NaN,0.0,NaN,2947.88,0.0,0.0,0.0,0.0,...,0,0,0,0:0:0,1900-01-01 00:00:00,10,44,15,15:44:10,1900-01-01 15:44:10
1356434,2947.93,NaN,NaN,0.0,NaN,2948.09,0.0,0.0,0.0,0.0,...,0,0,0,0:0:0,1900-01-01 00:00:00,11,44,15,15:44:11,1900-01-01 15:44:11
1356435,2948.11,NaN,NaN,0.0,NaN,2949.04,0.0,0.0,0.0,0.0,...,0,0,0,0:0:0,1900-01-01 00:00:00,11,44,15,15:44:11,1900-01-01 15:44:11


# Importing Video Data

- These CSVs are create in Bonsai when the video is being recorded. They give a time stamp of when the video is first being recorded.

In [299]:
bonsai_recording_start_files = glob.glob("./data/**/*csv")

In [300]:
bonsai_recording_start_files

['./data/bonsai_video_and_time_stamps/CAGE1_1_video_start_time2022-07-26T09_27_11.csv',
 './data/bonsai_video_and_time_stamps/CAGE2_2_video_start_time2022-07-24T10_21_52.csv']

In [361]:
video_recording_time_stamps = pd.read_csv(bonsai_recording_start_files[1], header=None)

In [362]:
video_recording_time_stamps.head()

,0,1
0,(7/24/2022 2:21:53 PM,186); 2022-07-24 10:21:09.865000
1,(7/24/2022 2:21:53 PM,218); 2022-07-24 10:21:09.896000
2,(7/24/2022 2:21:53 PM,250); 2022-07-24 10:21:09.929000
3,(7/24/2022 2:21:53 PM,284); 2022-07-24 10:21:09.963000
4,(7/24/2022 2:21:53 PM,315); 2022-07-24 10:21:09.994000


- The video is first being recorded in the time specified by the cell in the first row and the first column
    - We will try to get on the hour, minute, seconds from this
        - By getting everthing after the space, and then removing everything before the period

In [363]:
video_recording_time_stamps["time"] = video_recording_time_stamps[1].apply(lambda x: x.split()[-1].split(".")[0])


In [364]:
video_recording_time_stamps.head()

,0,1,time
0,(7/24/2022 2:21:53 PM,186); 2022-07-24 10:21:09.865000,10:21:09
1,(7/24/2022 2:21:53 PM,218); 2022-07-24 10:21:09.896000,10:21:09
2,(7/24/2022 2:21:53 PM,250); 2022-07-24 10:21:09.929000,10:21:09
3,(7/24/2022 2:21:53 PM,284); 2022-07-24 10:21:09.963000,10:21:09
4,(7/24/2022 2:21:53 PM,315); 2022-07-24 10:21:09.994000,10:21:09


- Getting the first time that's listed, which is when the video starts being recorded

In [365]:
video_recording_start_time_str = video_recording_time_stamps["time"][0]


In [366]:
video_recording_start_time_str

'10:21:09'

- Converting this into a datetime object so that we can make calculations

In [367]:
video_recording_start_time_object = datetime.strptime(video_recording_start_time_str, '%H:%M:%S')


In [368]:
video_recording_start_time_object

datetime.datetime(1900, 1, 1, 10, 21, 9)

# Syncing the MED-PC timestamps with the time in the video

- To get when in the video something happens(i.e. 10 seconds), we will calculate the difference of the time of the event with the time that the video first starts being recorded
- NOTE: Make sure that you are using the associated files for the MED-PC recording file and the video file

In [369]:
cs_time_df["file_path"].unique()

array(['./data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-22_12h26m_Subject 1.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-23_14h55m_Subject 4.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-23_10h06m_Subject 2.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-26_11h36m_Subject 4.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-22_10h11m_Subject 2.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-22_09h00m_Subject 3.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-26_09h05m_Subject 3.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-22_11h19m_Subject 3.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-23_14h55m_Subject 3.txt',
       './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-19_13h38m_Subject 1.txt',


In [370]:
all_med_pc_files = glob.glob("./data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-24*Subject 2*")

In [371]:
all_med_pc_files

['./data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-24_11h27m_Subject 2.txt',
 './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-24_12h35m_Subject 2.txt',
 './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-24_08h28m_Subject 2.txt',
 './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-24_13h47m_Subject 2.txt',
 './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-24_14h53m_Subject 2.txt',
 './data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-24_10h17m_Subject 2.txt']

In [372]:
first_medpc_recording_file = all_med_pc_files[-1]

first_cs_time_df = cs_time_df[cs_time_df["file_path"] == first_medpc_recording_file]

In [373]:
first_cs_time_df["file_path"].iloc[0]

'./data/pilot_c57_vs_cd1/reward_training/med_pc_text_files/2022-07-24_10h17m_Subject 2.txt'

In [374]:
# The time of the first controlled stimulus 
first_cs_time_df["cs_time_object"].iloc[0]

Timestamp('1900-01-01 10:22:35')

In [375]:
# The time of the second controlled stiumuls
first_cs_time_df["cs_time_object"].iloc[1]

Timestamp('1900-01-01 10:23:54')

In [376]:
# The time that the video started being recorded
video_recording_start_time_object

datetime.datetime(1900, 1, 1, 10, 21, 9)

In [377]:
# The time in the video that the controlled stimulus occurs
cs_01_time_object = first_cs_time_df["cs_time_object"].iloc[0] - video_recording_start_time_object 
cs_02_time_object = first_cs_time_df["cs_time_object"].iloc[1] - video_recording_start_time_object 
cs_03_time_object = first_cs_time_df["cs_time_object"].iloc[2] - video_recording_start_time_object 

In [378]:
cs_01_time_object.seconds

86

In [379]:
cs_02_time_object.seconds

165

In [380]:
cs_03_time_object.seconds

256

In [381]:
first_cs_time_df = first_cs_time_df[first_cs_time_df["(S)CSpresentation"] // 10 != 0].dropna()

In [382]:
first_cs_time_df["(S)CSpresentation"]

1295695      60.01
1295696     140.01
1295697     230.01
1295698     310.01
1295699     385.01
1295700     485.01
1295701     580.01
1295702     670.01
1295703     750.01
1295704     840.01
1295705     940.01
1295706    1030.01
1295707    1150.01
1295708    1240.01
1295709    1325.01
1295710    1415.01
1295711    1510.01
1295712    1630.01
1295713    1710.01
1295714    1805.01
1295715    1885.01
1295716    1975.01
1295717    2055.01
1295718    2130.01
1295719    2230.01
1295720    2325.01
1295721    2415.01
1295722    2495.01
1295723    2585.01
1295724    2685.01
1295725    2775.01
1295726    2895.01
1295727    2985.01
1295728    3070.01
1295729    3160.01
1295730    3255.01
1295731    3345.01
1295732    3425.01
1295733    3515.01
1295734    8000.00
Name: (S)CSpresentation, dtype: float64

In [383]:
first_cs_time_df["cs_video_time_stamp"] = first_cs_time_df["cs_time_object"] - video_recording_start_time_object 
first_cs_time_df["cs_video_time_seconds"] = first_cs_time_df["cs_video_time_stamp"].apply(lambda x: str(x.seconds))
first_cs_time_df["cs_video_time_string"] = first_cs_time_df["cs_video_time_stamp"].apply(lambda x: str(x.seconds // 60) + ":" + str(x.seconds % 60) )

In [384]:
first_cs_time_df["cs_video_time_seconds"].reset_index(drop=True)

0        86
1       165
2       256
3       335
4       411
5       511
6       605
7       696
8       776
9       866
10      965
11     1056
12     1175
13     1266
14     1351
15     1440
16     1536
17     1656
18     1735
19     1830
20     1911
21     2001
22     2080
23     2155
24     2256
25     2351
26     2441
27     2521
28     2611
29     2711
30     2801
31     2921
32     3010
33     3096
34     3186
35     3281
36     3371
37     3450
38     3540
39    49131
Name: cs_video_time_seconds, dtype: object

In [386]:
first_cs_time_df["cs_video_time_string"].reset_index(drop=True)

0       1:26
1       2:45
2       4:16
3       5:35
4       6:51
5       8:31
6       10:5
7      11:36
8      12:56
9      14:26
10      16:5
11     17:36
12     19:35
13      21:6
14     22:31
15      24:0
16     25:36
17     27:36
18     28:55
19     30:30
20     31:51
21     33:21
22     34:40
23     35:55
24     37:36
25     39:11
26     40:41
27      42:1
28     43:31
29     45:11
30     46:41
31     48:41
32     50:10
33     51:36
34      53:6
35     54:41
36     56:11
37     57:30
38      59:0
39    818:51
Name: cs_video_time_string, dtype: object

Time stamps from video for 7:26 cage 1 subject 1:
- 00:47, 1:54, 3:08, 4:16, 5:18, 6:41, 8:00, 9:14, 10:21, 11:36, 12:59, 14:14, 16:00, 17:08, 18:19, 19:38, 20:53, 22:32, 23:46, 24:58, 26:04, 27:19, 28:25, 29:28, 30:51, 32:10, 33:24, 34:31, 35:46, 37:09, 38:24, 40:04, 41:18, 42:29, 43:44, 45:03, 46:17, 47:24, 48:39, 50:40
Time stamps from video for 7/24 for cage 2 subject 2:
- 0:38, 1:30, 2:30, 3:24, 4:14, 5:20, 6:24, 7:23, 8:16, 9:16, 10:23, 11:23, 12:42, 13:42, 14:39, 15:39: 16:43, 18:03, 18:56, 19:59, 20:50, 21:53, 22:45, 23:35, 24:42, 25:45, 26:45, 27:38, 28:38, 29:45, 30:45, 32:04, 33:05, 34:01, 35:01, 36:04, 37:05, 37:57, 38:57, 40:34

# Syncing the MED-PC Port Entires with the time in the video

- To get when in the video something happens(i.e. 10 seconds), we will calculate the difference of the time of the event with the time that the video first starts being recorded
- NOTE: Make sure that you are using the associated files for the MED-PC recording file and the video file

In [330]:
first_cs_time_df["(F)port_entry_minutescomputer"].head(n=50)

1169866    26.0
1169867    26.0
1169868    27.0
1169869    27.0
1169870    27.0
1169871    27.0
1169872    27.0
1169873    27.0
1169874    27.0
1169875    27.0
1169876    27.0
1169877    27.0
1169878    28.0
1169879    28.0
1169880    28.0
1169881    28.0
1169882    28.0
1169883    28.0
1169884    28.0
1169885    28.0
1169886    28.0
1169887    28.0
1169888    28.0
1169889    28.0
1169890    28.0
1169891    28.0
1169892    28.0
1169893    28.0
1169894    28.0
1169895    28.0
1169896    28.0
1169897    28.0
1169898    28.0
1169899    29.0
1169900    29.0
1169901    29.0
1169902    29.0
1169903    29.0
1169904    29.0
1169905    29.0
Name: (F)port_entry_minutescomputer, dtype: float64

In [331]:
first_cs_time_df["(F)port_entry_minutescomputer"].tail(n=50)

1169866    26.0
1169867    26.0
1169868    27.0
1169869    27.0
1169870    27.0
1169871    27.0
1169872    27.0
1169873    27.0
1169874    27.0
1169875    27.0
1169876    27.0
1169877    27.0
1169878    28.0
1169879    28.0
1169880    28.0
1169881    28.0
1169882    28.0
1169883    28.0
1169884    28.0
1169885    28.0
1169886    28.0
1169887    28.0
1169888    28.0
1169889    28.0
1169890    28.0
1169891    28.0
1169892    28.0
1169893    28.0
1169894    28.0
1169895    28.0
1169896    28.0
1169897    28.0
1169898    28.0
1169899    29.0
1169900    29.0
1169901    29.0
1169902    29.0
1169903    29.0
1169904    29.0
1169905    29.0
Name: (F)port_entry_minutescomputer, dtype: float64

In [332]:
first_cs_time_df["(B)port_entry_secondscomputer"].head(n=50)

1169866    26.0
1169867    26.0
1169868    14.0
1169869    34.0
1169870    38.0
1169871    39.0
1169872    39.0
1169873    42.0
1169874    42.0
1169875    49.0
1169876    54.0
1169877    57.0
1169878     6.0
1169879     7.0
1169880     7.0
1169881     8.0
1169882     9.0
1169883     9.0
1169884     9.0
1169885     9.0
1169886    29.0
1169887    31.0
1169888    48.0
1169889    48.0
1169890    48.0
1169891    48.0
1169892    48.0
1169893    48.0
1169894    48.0
1169895    48.0
1169896    48.0
1169897    49.0
1169898    59.0
1169899     3.0
1169900     4.0
1169901     4.0
1169902    11.0
1169903    38.0
1169904    38.0
1169905    43.0
Name: (B)port_entry_secondscomputer, dtype: float64

In [333]:
first_cs_time_df["(B)port_entry_secondscomputer"].tail(n=50)

1169866    26.0
1169867    26.0
1169868    14.0
1169869    34.0
1169870    38.0
1169871    39.0
1169872    39.0
1169873    42.0
1169874    42.0
1169875    49.0
1169876    54.0
1169877    57.0
1169878     6.0
1169879     7.0
1169880     7.0
1169881     8.0
1169882     9.0
1169883     9.0
1169884     9.0
1169885     9.0
1169886    29.0
1169887    31.0
1169888    48.0
1169889    48.0
1169890    48.0
1169891    48.0
1169892    48.0
1169893    48.0
1169894    48.0
1169895    48.0
1169896    48.0
1169897    49.0
1169898    59.0
1169899     3.0
1169900     4.0
1169901     4.0
1169902    11.0
1169903    38.0
1169904    38.0
1169905    43.0
Name: (B)port_entry_secondscomputer, dtype: float64

In [334]:
# The time in the video that the controlled stimulus occurs
port_entry_01_time_object = first_cs_time_df["pe_time_object"].iloc[62] - video_recording_start_time_object 
port_entry_02_time_object = first_cs_time_df["pe_time_object"].iloc[74] - video_recording_start_time_object 
port_entry_03_time_object = first_cs_time_df["pe_time_object"].iloc[82] - video_recording_start_time_object 

IndexError: single positional indexer is out-of-bounds

In [203]:
first_cs_time_df["pe_time_object"].iloc[62]

Timestamp('1900-01-01 12:48:58')

In [195]:
port_entry_01_time_object.seconds

341

In [196]:
port_entry_02_time_object.seconds

353

In [197]:
port_entry_03_time_object.seconds

359

# Trimming the video

In [177]:
video_object = VideoFileClip("./data/automatic_30_min_end_test/2clock_test_recording_2022-07-12T12_43_17.avi")

In [178]:
video_object.duration

1910.47

fixed func issue with: https://github.com/Zulko/moviepy/issues/1765


In [179]:
cs1_trimmed_video = video_object.subclip(cs_01_time_object.seconds, cs_02_time_object.seconds + 20)

In [180]:
cs2_trimmed_video = video_object.subclip(cs_02_time_object.seconds, cs_03_time_object.seconds + 20)

In [181]:
cs3_trimmed_video = video_object.subclip(cs_03_time_object.seconds, video_object.duration)

In [182]:
pe1_trimmed_video = video_object.subclip(port_entry_01_time_object.seconds, port_entry_02_time_object.seconds + 20)

In [183]:
pe2_trimmed_video = video_object.subclip(port_entry_02_time_object.seconds, port_entry_03_time_object.seconds + 20)

In [198]:
pe3_trimmed_video = video_object.subclip(port_entry_03_time_object.seconds, video_object.duration)

In [185]:
cs1_trimmed_video.to_videofile("./data/automatic_30_min_end_test/trimmed_cs1.mp4")

                                                                                                                           
t:   0%|                                                                 | 2/47535 [11:18<4476:49:22, 339.06s/it, now=None]
                                                                                                                           
t:  74%|████████████████████████████████████████████████▊                 | 34736/47025 [09:32<00:20, 596.82it/s, now=None]

Moviepy - Building video ./data/automatic_30_min_end_test/trimmed_cs1.mp4.
Moviepy - Writing video ./data/automatic_30_min_end_test/trimmed_cs1.mp4





t:   0%|                                                                                | 0/3000 [00:00<?, ?it/s, now=None]

t:   2%|█                                                                     | 48/3000 [00:00<00:07, 369.16it/s, now=None]

t:   3%|██▏                                                                   | 95/3000 [00:00<00:06, 421.08it/s, now=None]

t:   5%|███▏                                                                 | 140/3000 [00:00<00:06, 422.67it/s, now=None]

t:   6%|████▎                                                                | 186/3000 [00:00<00:06, 436.37it/s, now=None]

t:   8%|█████▎                                                               | 233/3000 [00:00<00:06, 446.35it/s, now=None]

t:   9%|██████▍                                                              | 279/3000 [00:00<00:06, 450.21it/s, now=None]

t:  11%|███████▍                                                             | 326/3000 [00:00<00:05, 453.68it/s, now=None]

Moviepy - Done !
Moviepy - video ready ./data/automatic_30_min_end_test/trimmed_cs1.mp4


In [186]:
cs2_trimmed_video.to_videofile("./data/automatic_30_min_end_test/trimmed_cs2.mp4")

                                                                                                                           
t:   0%|                                                                 | 2/47535 [11:24<4517:45:08, 342.16s/it, now=None]
                                                                                                                           
t:  74%|████████████████████████████████████████████████▊                 | 34736/47025 [09:38<00:20, 596.82it/s, now=None]

Moviepy - Building video ./data/automatic_30_min_end_test/trimmed_cs2.mp4.
Moviepy - Writing video ./data/automatic_30_min_end_test/trimmed_cs2.mp4





t:   0%|                                                                                | 0/3300 [00:00<?, ?it/s, now=None]

t:   1%|▉                                                                     | 46/3300 [00:00<00:07, 459.41it/s, now=None]

t:   3%|█▉                                                                    | 92/3300 [00:00<00:08, 385.45it/s, now=None]

t:   4%|██▉                                                                  | 138/3300 [00:00<00:07, 416.01it/s, now=None]

t:   5%|███▊                                                                 | 181/3300 [00:00<00:07, 407.02it/s, now=None]

t:   7%|████▋                                                                | 223/3300 [00:00<00:07, 409.21it/s, now=None]

t:   8%|█████▌                                                               | 266/3300 [00:00<00:07, 412.12it/s, now=None]

t:   9%|██████▍                                                              | 308/3300 [00:00<00:07, 414.18it/s, now=None]

Moviepy - Done !
Moviepy - video ready ./data/automatic_30_min_end_test/trimmed_cs2.mp4


In [187]:
cs3_trimmed_video.to_videofile("./data/automatic_30_min_end_test//trimmed_cs3.mp4")

                                                                                                                           
t:   0%|                                                                 | 2/47535 [11:30<4558:12:27, 345.22s/it, now=None]
                                                                                                                           
t:  74%|████████████████████████████████████████████████▊                 | 34736/47025 [09:44<00:20, 596.82it/s, now=None]

Moviepy - Building video ./data/automatic_30_min_end_test//trimmed_cs3.mp4.
Moviepy - Writing video ./data/automatic_30_min_end_test//trimmed_cs3.mp4





t:   0%|                                                                               | 0/47535 [00:00<?, ?it/s, now=None]

t:   0%|                                                                     | 42/47535 [00:00<01:53, 419.86it/s, now=None]

t:   0%|                                                                     | 84/47535 [00:00<02:05, 378.62it/s, now=None]

t:   0%|▏                                                                   | 128/47535 [00:00<01:57, 403.23it/s, now=None]

t:   0%|▏                                                                   | 171/47535 [00:00<01:58, 398.46it/s, now=None]

t:   0%|▎                                                                   | 214/47535 [00:00<01:57, 403.09it/s, now=None]

t:   1%|▎                                                                   | 260/47535 [00:00<01:53, 416.40it/s, now=None]

t:   1%|▍                                                                   | 311/47535 [00:00<01:46, 444.84it/s, now=None]

t:  16%|██████████▉                                                        | 7743/47535 [00:12<01:00, 662.20it/s, now=None]

t:  16%|███████████                                                        | 7810/47535 [00:12<01:00, 661.73it/s, now=None]

t:  17%|███████████                                                        | 7879/47535 [00:13<00:59, 669.58it/s, now=None]

t:  17%|███████████▏                                                       | 7947/47535 [00:13<00:58, 671.33it/s, now=None]

t:  17%|███████████▎                                                       | 8015/47535 [00:13<00:59, 668.79it/s, now=None]

t:  17%|███████████▍                                                       | 8082/47535 [00:13<00:59, 667.04it/s, now=None]

t:  17%|███████████▍                                                       | 8150/47535 [00:13<00:58, 668.91it/s, now=None]

t:  17%|███████████▌                                                       | 8218/47535 [00:13<00:58, 671.33it/s, now=None]



t:  33%|██████████████████████                                            | 15890/47535 [00:25<00:51, 615.87it/s, now=None]

t:  34%|██████████████████████▏                                           | 15954/47535 [00:25<00:50, 620.87it/s, now=None]

t:  34%|██████████████████████▏                                           | 16019/47535 [00:25<00:50, 627.92it/s, now=None]

t:  34%|██████████████████████▎                                           | 16082/47535 [00:25<00:50, 626.83it/s, now=None]

t:  34%|██████████████████████▍                                           | 16146/47535 [00:25<00:49, 629.06it/s, now=None]

t:  34%|██████████████████████▌                                           | 16209/47535 [00:26<00:49, 629.32it/s, now=None]

t:  34%|██████████████████████▌                                           | 16272/47535 [00:26<00:50, 625.16it/s, now=None]

t:  34%|██████████████████████▋                                           | 16335/47535 [00:26<00:50, 614.83it/s, now=None]



t:  50%|████████████████████████████████▉                                 | 23689/47535 [00:38<00:41, 580.74it/s, now=None]

t:  50%|████████████████████████████████▉                                 | 23750/47535 [00:38<00:40, 588.90it/s, now=None]

t:  50%|█████████████████████████████████                                 | 23809/47535 [00:38<00:40, 581.96it/s, now=None]

t:  50%|█████████████████████████████████▏                                | 23870/47535 [00:38<00:40, 587.25it/s, now=None]

t:  50%|█████████████████████████████████▏                                | 23929/47535 [00:38<00:40, 585.21it/s, now=None]

t:  50%|█████████████████████████████████▎                                | 23990/47535 [00:38<00:39, 590.45it/s, now=None]

t:  51%|█████████████████████████████████▍                                | 24050/47535 [00:38<00:40, 582.00it/s, now=None]

t:  51%|█████████████████████████████████▍                                | 24112/47535 [00:38<00:39, 589.32it/s, now=None]



t:  66%|███████████████████████████████████████████▌                      | 31387/47535 [00:51<00:28, 573.91it/s, now=None]

t:  66%|███████████████████████████████████████████▋                      | 31448/47535 [00:51<00:27, 583.57it/s, now=None]

t:  66%|███████████████████████████████████████████▋                      | 31508/47535 [00:51<00:27, 585.14it/s, now=None]

t:  66%|███████████████████████████████████████████▊                      | 31567/47535 [00:51<00:27, 581.50it/s, now=None]

t:  67%|███████████████████████████████████████████▉                      | 31626/47535 [00:51<00:27, 581.91it/s, now=None]

t:  67%|███████████████████████████████████████████▉                      | 31685/47535 [00:51<00:27, 576.76it/s, now=None]

t:  67%|████████████████████████████████████████████                      | 31746/47535 [00:51<00:26, 585.93it/s, now=None]

t:  67%|████████████████████████████████████████████▏                     | 31805/47535 [00:51<00:27, 562.96it/s, now=None]



t:  82%|██████████████████████████████████████████████████████▎           | 39162/47535 [01:03<00:13, 609.79it/s, now=None]

t:  83%|██████████████████████████████████████████████████████▍           | 39224/47535 [01:03<00:13, 611.87it/s, now=None]

t:  83%|██████████████████████████████████████████████████████▌           | 39286/47535 [01:03<00:13, 613.00it/s, now=None]

t:  83%|██████████████████████████████████████████████████████▋           | 39348/47535 [01:04<00:13, 602.36it/s, now=None]

t:  83%|██████████████████████████████████████████████████████▋           | 39411/47535 [01:04<00:13, 606.40it/s, now=None]

t:  83%|██████████████████████████████████████████████████████▊           | 39476/47535 [01:04<00:13, 618.96it/s, now=None]

t:  83%|██████████████████████████████████████████████████████▉           | 39540/47535 [01:04<00:12, 623.59it/s, now=None]

t:  83%|██████████████████████████████████████████████████████▉           | 39603/47535 [01:04<00:13, 601.35it/s, now=None]



t:  99%|█████████████████████████████████████████████████████████████████ | 46844/47535 [01:16<00:01, 603.24it/s, now=None]

t:  99%|█████████████████████████████████████████████████████████████████▏| 46908/47535 [01:16<00:01, 613.12it/s, now=None]

t:  99%|█████████████████████████████████████████████████████████████████▏| 46972/47535 [01:16<00:00, 620.92it/s, now=None]

t:  99%|█████████████████████████████████████████████████████████████████▎| 47035/47535 [01:16<00:00, 620.86it/s, now=None]

t:  99%|█████████████████████████████████████████████████████████████████▍| 47098/47535 [01:16<00:00, 612.47it/s, now=None]

t:  99%|█████████████████████████████████████████████████████████████████▍| 47162/47535 [01:16<00:00, 619.67it/s, now=None]

t:  99%|█████████████████████████████████████████████████████████████████▌| 47226/47535 [01:17<00:00, 625.60it/s, now=None]

t:  99%|█████████████████████████████████████████████████████████████████▋| 47289/47535 [01:17<00:00, 626.70it/s, now=None]



Moviepy - Done !
Moviepy - video ready ./data/automatic_30_min_end_test//trimmed_cs3.mp4


In [188]:
pe1_trimmed_video = video_object.subclip(port_entry_01_time_object.seconds, port_entry_02_time_object.seconds + 20)

In [189]:
pe2_trimmed_video = video_object.subclip(port_entry_02_time_object.seconds, port_entry_03_time_object.seconds + 20)

In [199]:
pe3_trimmed_video = video_object.subclip(port_entry_03_time_object.seconds, video_object.duration)

In [191]:
pe1_trimmed_video.to_videofile("./data/automatic_30_min_end_test/trimmed_pe1.mp4")

                                                                                                                           
t:   0%|                                                                 | 2/47535 [12:48<5071:59:46, 384.14s/it, now=None]
                                                                                                                           
t:  74%|████████████████████████████████████████████████▊                 | 34736/47025 [11:02<00:20, 596.82it/s, now=None]

Moviepy - Building video ./data/automatic_30_min_end_test/trimmed_pe1.mp4.
Moviepy - Writing video ./data/automatic_30_min_end_test/trimmed_pe1.mp4





t:   0%|                                                                                 | 0/960 [00:00<?, ?it/s, now=None]

t:   4%|███                                                                    | 42/960 [00:00<00:02, 418.63it/s, now=None]

t:   9%|██████▏                                                                | 84/960 [00:00<00:02, 417.68it/s, now=None]

t:  14%|█████████▉                                                            | 136/960 [00:00<00:01, 462.60it/s, now=None]

t:  20%|█████████████▉                                                        | 191/960 [00:00<00:01, 494.99it/s, now=None]

t:  26%|█████████████████▉                                                    | 246/960 [00:00<00:01, 514.49it/s, now=None]

t:  31%|█████████████████████▋                                                | 298/960 [00:00<00:01, 504.76it/s, now=None]

t:  36%|█████████████████████████▍                                            | 349/960 [00:00<00:01, 503.14it/s, now=None]

Moviepy - Done !
Moviepy - video ready ./data/automatic_30_min_end_test/trimmed_pe1.mp4


In [192]:
pe2_trimmed_video.to_videofile("./data/automatic_30_min_end_test/trimmed_pe2.mp4")

                                                                                                                           
t:   0%|                                                                 | 2/47535 [12:50<5085:37:08, 385.17s/it, now=None]
                                                                                                                           
t:  74%|████████████████████████████████████████████████▊                 | 34736/47025 [11:04<00:20, 596.82it/s, now=None]

Moviepy - Building video ./data/automatic_30_min_end_test/trimmed_pe2.mp4.
Moviepy - Writing video ./data/automatic_30_min_end_test/trimmed_pe2.mp4





t:   0%|                                                                                 | 0/870 [00:00<?, ?it/s, now=None]

t:   5%|███▊                                                                   | 46/870 [00:00<00:01, 459.56it/s, now=None]

t:  11%|███████▌                                                               | 92/870 [00:00<00:01, 426.30it/s, now=None]

t:  16%|███████████▎                                                          | 140/870 [00:00<00:01, 442.34it/s, now=None]

t:  22%|███████████████▋                                                      | 195/870 [00:00<00:01, 482.83it/s, now=None]

t:  28%|███████████████████▋                                                  | 244/870 [00:00<00:01, 463.66it/s, now=None]

t:  33%|███████████████████████▍                                              | 291/870 [00:00<00:01, 465.23it/s, now=None]

t:  39%|███████████████████████████▏                                          | 338/870 [00:00<00:01, 437.95it/s, now=None]

Moviepy - Done !
Moviepy - video ready ./data/automatic_30_min_end_test/trimmed_pe2.mp4


In [200]:
pe3_trimmed_video.to_videofile("./data/automatic_30_min_end_test//trimmed_pe3.mp4")

                                                                                                                           
t:   0%|                                                             | 2/47535 [1:34:25<37403:05:58, 2832.79s/it, now=None]
                                                                                                                           
t:  74%|███████████████████████████████████████████████▎                | 34736/47025 [1:32:39<00:20, 596.82it/s, now=None]

Moviepy - Building video ./data/automatic_30_min_end_test//trimmed_pe3.mp4.
Moviepy - Writing video ./data/automatic_30_min_end_test//trimmed_pe3.mp4





t:   0%|                                                                               | 0/46545 [00:00<?, ?it/s, now=None]

t:   0%|                                                                     | 48/46545 [00:00<01:41, 459.74it/s, now=None]

t:   0%|▏                                                                   | 102/46545 [00:00<01:32, 502.74it/s, now=None]

t:   0%|▏                                                                   | 153/46545 [00:00<01:38, 472.94it/s, now=None]

t:   0%|▎                                                                   | 201/46545 [00:00<01:43, 447.83it/s, now=None]

t:   1%|▎                                                                   | 255/46545 [00:00<01:36, 478.76it/s, now=None]

t:   1%|▍                                                                   | 314/46545 [00:00<01:29, 514.41it/s, now=None]

t:   1%|▌                                                                   | 374/46545 [00:00<01:25, 539.44it/s, now=None]

t:  17%|███████████▍                                                       | 7936/46545 [00:12<01:04, 601.60it/s, now=None]

t:  17%|███████████▌                                                       | 8000/46545 [00:12<01:02, 612.13it/s, now=None]

t:  17%|███████████▌                                                       | 8067/46545 [00:13<01:01, 627.42it/s, now=None]

t:  17%|███████████▋                                                       | 8131/46545 [00:13<01:01, 623.37it/s, now=None]

t:  18%|███████████▊                                                       | 8194/46545 [00:13<01:04, 591.31it/s, now=None]

t:  18%|███████████▉                                                       | 8257/46545 [00:13<01:03, 600.52it/s, now=None]

t:  18%|███████████▉                                                       | 8321/46545 [00:13<01:02, 611.11it/s, now=None]

t:  18%|████████████                                                       | 8386/46545 [00:13<01:01, 616.10it/s, now=None]



t:  35%|██████████████████████▉                                           | 16201/46545 [00:25<00:46, 653.41it/s, now=None]

t:  35%|███████████████████████                                           | 16272/46545 [00:25<00:45, 669.13it/s, now=None]

t:  35%|███████████████████████▏                                          | 16340/46545 [00:25<00:45, 660.35it/s, now=None]

t:  35%|███████████████████████▎                                          | 16408/46545 [00:25<00:45, 665.29it/s, now=None]

t:  35%|███████████████████████▎                                          | 16475/46545 [00:26<00:45, 664.01it/s, now=None]

t:  36%|███████████████████████▍                                          | 16544/46545 [00:26<00:44, 669.99it/s, now=None]

t:  36%|███████████████████████▌                                          | 16612/46545 [00:26<00:44, 665.27it/s, now=None]

t:  36%|███████████████████████▋                                          | 16679/46545 [00:26<00:44, 665.53it/s, now=None]



t:  52%|██████████████████████████████████▎                               | 24184/46545 [00:38<00:39, 568.82it/s, now=None]

t:  52%|██████████████████████████████████▎                               | 24241/46545 [00:38<00:39, 564.84it/s, now=None]

t:  52%|██████████████████████████████████▍                               | 24302/46545 [00:38<00:38, 577.51it/s, now=None]

t:  52%|██████████████████████████████████▌                               | 24365/46545 [00:38<00:37, 587.70it/s, now=None]

t:  52%|██████████████████████████████████▋                               | 24426/46545 [00:38<00:37, 593.03it/s, now=None]

t:  53%|██████████████████████████████████▋                               | 24486/46545 [00:39<00:37, 588.02it/s, now=None]

t:  53%|██████████████████████████████████▊                               | 24545/46545 [00:39<00:37, 587.98it/s, now=None]

t:  53%|██████████████████████████████████▉                               | 24607/46545 [00:39<00:37, 590.68it/s, now=None]



t:  68%|████████████████████████████████████████████▉                     | 31686/46545 [00:51<00:26, 567.05it/s, now=None]

t:  68%|█████████████████████████████████████████████                     | 31743/46545 [00:51<00:26, 564.61it/s, now=None]

t:  68%|█████████████████████████████████████████████                     | 31800/46545 [00:51<00:26, 557.78it/s, now=None]

t:  68%|█████████████████████████████████████████████▏                    | 31859/46545 [00:51<00:25, 565.04it/s, now=None]

t:  69%|█████████████████████████████████████████████▎                    | 31917/46545 [00:51<00:25, 566.64it/s, now=None]

t:  69%|█████████████████████████████████████████████▎                    | 31976/46545 [00:51<00:25, 572.37it/s, now=None]

t:  69%|█████████████████████████████████████████████▍                    | 32034/46545 [00:51<00:25, 574.18it/s, now=None]

t:  69%|█████████████████████████████████████████████▌                    | 32094/46545 [00:52<00:24, 580.00it/s, now=None]



t:  84%|███████████████████████████████████████████████████████▌          | 39153/46545 [01:04<00:13, 560.00it/s, now=None]

t:  84%|███████████████████████████████████████████████████████▌          | 39216/46545 [01:04<00:12, 578.64it/s, now=None]

t:  84%|███████████████████████████████████████████████████████▋          | 39274/46545 [01:04<00:12, 573.98it/s, now=None]

t:  85%|███████████████████████████████████████████████████████▊          | 39332/46545 [01:04<00:12, 567.22it/s, now=None]

t:  85%|███████████████████████████████████████████████████████▊          | 39392/46545 [01:04<00:12, 575.55it/s, now=None]

t:  85%|███████████████████████████████████████████████████████▉          | 39450/46545 [01:04<00:12, 563.28it/s, now=None]

t:  85%|████████████████████████████████████████████████████████          | 39512/46545 [01:04<00:12, 578.97it/s, now=None]

t:  85%|████████████████████████████████████████████████████████          | 39570/46545 [01:04<00:12, 578.59it/s, now=None]



t:   0%|                                                             | 2/47535 [1:35:42<37911:51:00, 2871.32s/it, now=None]
                                                                                                                           
t:  74%|███████████████████████████████████████████████▎                | 34736/47025 [1:33:56<00:20, 596.82it/s, now=None]

Moviepy - Done !
Moviepy - video ready ./data/automatic_30_min_end_test//trimmed_pe3.mp4


In [56]:
Video("./data/automatic_30_min_end_test/trimmed_cs1.mp4")